##Importing libraries, Data Loading and Exploration

In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import keras
from IPython.display import display, Image
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from matplotlib.pyplot import imread
import matplotlib.pyplot as plt
import pandas as pd
from google.colab import drive
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, Model, Input
from keras.layers import AveragePooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
from keras.applications.imagenet_utils import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import os

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip "drive/My Drive/Data/dog-breed-identification.zip" -d "drive/My Drive/Data/"

In [4]:
labels_csv = pd.read_csv("drive/My Drive/Data/labels.csv")
print(labels_csv.describe())
print(labels_csv.head())

                                      id               breed
count                              10222               10222
unique                             10222                 120
top     000bec180eb18c7604dcecc8fe0dba07  scottish_deerhound
freq                                   1                 126
                                 id             breed
0  000bec180eb18c7604dcecc8fe0dba07       boston_bull
1  001513dfcb2ffafc82cccf4d8bbaba97             dingo
2  001cdf01b096e06d78e9e5112d419397          pekinese
3  00214f311d5d2247d5dfe4fe24b2303d          bluetick
4  0021f9ceb3235effd7fcde7f7538ed62  golden_retriever


In [5]:
filenames = ["drive/My Drive/Data/train/" + fname + ".jpg" for fname in labels_csv["id"]]

In [6]:
labels = labels_csv["breed"].to_numpy()

In [7]:
unique_breeds = np.unique(labels)
len(unique_breeds)

120

In [8]:
boolean_labels = [label == np.array(unique_breeds) for label in labels]

In [9]:
X = filenames
y = boolean_labels

In [10]:
number_of_img = 1000

In [11]:
X_train, X_val, y_train, y_val = train_test_split(X[:number_of_img],
                                                  y[:number_of_img],
                                                  test_size=0.2,
                                                  random_state=42)

len(X_train), len(y_train), len(X_val), len(y_val)

(800, 800, 200, 200)

In [12]:
image = imread(filenames[30])
image.shape

(338, 450, 3)

##Data Preprocessing

In [13]:
imagesize = 224

def process_image(image_path):

  image = tf.io.read_file(image_path)
  image = tf.image.decode_jpeg(image, channels=3)
  image = tf.image.convert_image_dtype(image, tf.float32)
  image = tf.image.resize(image, size=[imagesize, imagesize])
  return image

In [14]:
def get_image_label(image_path, label):
  image = process_image(image_path)
  return image, label

In [15]:
batchsize = 32

def create_data_batches(x, y=None, batch_size=batchsize, valid_data=False, test_data=False):

  if test_data:
    data = tf.data.Dataset.from_tensor_slices((tf.constant(x)))
    data_batch = data.map(process_image).batch(batchsize)
    return data_batch


  elif valid_data:
    data = tf.data.Dataset.from_tensor_slices((tf.constant(x),
                                               tf.constant(y)))
    data_batch = data.map(get_image_label).batch(batchsize)
    return data_batch

  else:
    data = tf.data.Dataset.from_tensor_slices((tf.constant(x),
                                              tf.constant(y)))

    data = data.shuffle(buffer_size=len(x))

    data = data.map(get_image_label)

    data_batch = data.batch(batchsize)
  return data_batch

In [16]:
train_data = create_data_batches(X_train, y_train)
val_data = create_data_batches(X_val, y_val, valid_data=True)

##Model Building

In [17]:
inputshape = [None, imagesize, imagesize, 3]

outputshape = len(unique_breeds)

modelurl = "https://tfhub.dev/google/bit/m-r50x1/1"
MODULE = hub.KerasLayer(modelurl)

In [18]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_accuracy",
                                                  patience=3)

In [19]:
def create_model(input_shape=inputshape, output_shape=outputshape, model_url=modelurl):
  print("Building model with:", modelurl)


  model = tf.keras.Sequential([
    hub.KerasLayer(modelurl),
    tf.keras.layers.Dense(units=outputshape,
                          activation="softmax")
  ])

  model.compile(
      loss=tf.keras.losses.BinaryCrossentropy(),
      optimizer=tf.keras.optimizers.Adam(),
      metrics=["accuracy"]
  )

  # Build the model
  model.build(inputshape)

  return model

In [20]:
def train_model():


  model.fit(x=train_data,
            epochs=numepochs,
            validation_data=val_data,
            validation_freq=1,
            callbacks=[early_stopping])

  return model

In [21]:
model = create_model()
model.summary()

Building model with: https://tfhub.dev/google/bit/m-r50x1/1
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_1 (KerasLayer)  (None, 2048)              23500352  
                                                                 
 dense (Dense)               (None, 120)               245880    
                                                                 
Total params: 23,746,232
Trainable params: 245,880
Non-trainable params: 23,500,352
_________________________________________________________________


In [22]:
numepochs = 100

In [23]:
model = train_model()

Epoch 1/100
25/25 [==============================] - 33s 340ms/step - loss: 0.1928 - accuracy: 0.0288 - val_loss: 0.1204 - val_accuracy: 0.0900
Epoch 2/100
25/25 [==============================] - 8s 302ms/step - loss: 0.0900 - accuracy: 0.1800 - val_loss: 0.0824 - val_accuracy: 0.1700
Epoch 3/100
25/25 [==============================] - 7s 296ms/step - loss: 0.0494 - accuracy: 0.3562 - val_loss: 0.0525 - val_accuracy: 0.2650
Epoch 4/100
25/25 [==============================] - 6s 243ms/step - loss: 0.0253 - accuracy: 0.6363 - val_loss: 0.0412 - val_accuracy: 0.3350
Epoch 5/100
25/25 [==============================] - 7s 297ms/step - loss: 0.0150 - accuracy: 0.7600 - val_loss: 0.0339 - val_accuracy: 0.4250
Epoch 6/100
25/25 [==============================] - 6s 242ms/step - loss: 0.0098 - accuracy: 0.8725 - val_loss: 0.0301 - val_accuracy: 0.4350
Epoch 7/100
25/25 [==============================] - 6s 244ms/step - loss: 0.0069 - accuracy: 0.9375 - val_loss: 0.0279 - val_accuracy: 0.455

##Evaluation

In [24]:
model.evaluate(val_data)

7/7 [==============================] - 1s 170ms/step - loss: 0.0267 - accuracy: 0.5500


[0.026692228391766548, 0.550000011920929]

##Training on Full Data

In [25]:
full_data = create_data_batches(X, y)

In [26]:
full_model = create_model()
full_model.summary()

Building model with: https://tfhub.dev/google/bit/m-r50x1/1
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_2 (KerasLayer)  (None, 2048)              23500352  
                                                                 
 dense_1 (Dense)             (None, 120)               245880    
                                                                 
Total params: 23,746,232
Trainable params: 245,880
Non-trainable params: 23,500,352
_________________________________________________________________


In [27]:
full_model_early_stopping = tf.keras.callbacks.EarlyStopping(monitor="accuracy",
                                                             patience=3)

In [28]:
full_model.fit(x=full_data,
               epochs=numepochs,
               callbacks=[full_model_early_stopping])

Epoch 1/100
320/320 [==============================] - 81s 202ms/step - loss: 0.0440 - accuracy: 0.4318
Epoch 2/100
320/320 [==============================] - 64s 200ms/step - loss: 0.0146 - accuracy: 0.7222
Epoch 3/100
320/320 [==============================] - 65s 202ms/step - loss: 0.0107 - accuracy: 0.8073
Epoch 4/100
320/320 [==============================] - 65s 204ms/step - loss: 0.0081 - accuracy: 0.8603
Epoch 5/100
320/320 [==============================] - 65s 202ms/step - loss: 0.0068 - accuracy: 0.8925
Epoch 6/100
320/320 [==============================] - 65s 204ms/step - loss: 0.0057 - accuracy: 0.9138
Epoch 7/100
320/320 [==============================] - 65s 202ms/step - loss: 0.0044 - accuracy: 0.9412
Epoch 8/100
320/320 [==============================] - 65s 201ms/step - loss: 0.0037 - accuracy: 0.9531
Epoch 9/100
320/320 [==============================] - 64s 200ms/step - loss: 0.0030 - accuracy: 0.9670
Epoch 10/100
320/320 [==============================] - 65s 203m